In [ ]:
# Compression Lab

Due Tuesday, April 5 at 8 AM

## Introduction

The goal of compression is to represent a file using fewer bits. Compression was obviously very important many years ago when hard drive storage was measured in megabytes, not gigabytes or terabytes. But it remains relevant today in the age of small personal devices and big data. If the music on your phone were stored as uncompressed WAV files, you wouldn't be able to fit much music; fortunately, MP3 files are often about 10 times smaller. At the other extreme, even data centers that can store petabytes of data are struggling to keep up with satellites that are beaming terabytes of data back to Earth every second. Compression remains a relevant problem in the modern world.

There are two kinds of compression: lossy and lossless. Lossy compression is straightforward; to achieve a smaller file, we throw away some (hopefully less important) data. For example, MP3 leverages research on human psychoacoustics to discard frequencies that we wouldn't be able to hear anyway. On the other hand, lossless compression seems too good to be true. How can it be possible to achieve a smaller file without losing anything at all? We explore this question in this lab.

# ## Huffman Coding

In a uniform coding scheme, we use an equal number of bits to represent 
every symbol. For example, if there were 5 symbols in our world, a uniform
coding scheme might use a 3-bit code for each symbol:

A | B | C | D | E 
:--:|:--:|:--:|:--:|:--:
000 | 001 | 010 | 011 | 100

A table like this one that maps symbols to their codes is called a *codebook*.

But not all symbols are equally common. Suppose the actual probabilities 
of each letter were

A | B | C | D | E 
:--:|:--:|:--:|:--:|:--:
.05 | .15 | .20 | .25 | .35

If we used a shorter code to represent E, maybe we could have a shorter 
text on average. Huffman coding does exactly this. It constructs a code for 
each symbol using the following algorithm:

1. Find the two symbols with the smallest probabilities. Add a 1 to 
the beginning of the code for one symbol and a 0 to 
the beginning of the code for the other. (So far, the code for A is "1" and 
the code for B is "0".)
2. "Merge" those two symbols, and treat this merged symbol as a 
single symbol whose probability is the sum of the probabilities of the two 
symbols you merged. Repeat Steps 
1 and 2. (We now have a symbol AB whose probability is $.05 + .15 = .2$.
The two symbols now with the smallest probabilities are now 
AB and C. So we prepend a 1 to the beginning of the codes for A and B 
and a 0 to the beginning of the code for C. So the code for A becomes "11", 
the code for B becomes "10", and the code for C becomes "0".)
3. Repeat until all the symbols have been merged together.

To check your understanding of Huffman coding, follow the instructions 
above and make sure you get a codebook that looks like:

A | B | C | D | E 
:--:|:--:|:--:|:--:|:--:
111 | 110 | 10 | 01 | 00

(You might get something slightly different depending on which symbol you 
assigned a 1 to and which symbol you assigned a 0 to, at each step. As long as 
your codes are all of the same length, you should be okay.)

Let $L$ represent the length of the code for a randomly chosen symbol. Under 
uniform coding, $L=3$. Under Huffman coding:
$$ \textrm{E}[L] = .05(3) + .15(3) + .2(2) + .25(2) + .35(2) = 2.4. $$

In this lab, you will implement a simple Huffman coder. The first function has been implemented for you. You will have to implement three others.

In [83]:
def get_character_counts(text):
    """Count the number of occurrences of each symbol in a text.                                  
                                                                                                  
    Arguments:                                                                                    
    text -- string of text                                                                        
                                                                                                  
    Output:                                                                                       
    a dict that maps symbols to the number of times they appear                                   
      in the text                                                                                 
    """
    counts = {}
    # iterate over the characters in our text                                                     
    for char in text:
        # if this character is not already in our dict, add it                                    
        # (with a count of 1)                                                                     
        if char not in counts:
            counts[char] = 1
        # otherwise, add 1 to its count                                                           
        else:
            counts[char] += 1
    return counts

In [101]:
def generate_codebook(character_counts):
    """Uses Huffman coding to generate a codebook from character 
    counts / probabilities.

    Arguments:
    character_counts -- a dict that maps each character to either 
      its count or its probability.
                                                                                                  
    Output:                                                                                       
    a dict that maps symbols to a code consisting of 0's and 1's                                      
    """

    # initialize codebook with the same characters as character_counts                                   
    codebook = {}
    for symbol in character_counts.keys():
        codebook[symbol] = ""

    # YOUR CODE HERE
    finished = ""
    def sort_fun(x): ## need a key function for sorting
       return x[1]
    character_list = list(character_counts.items())
    while len(character_list) > 1:
       key_finished = 0
       character_list.sort(key=sort_fun) # sort the list in ascending order
       tempchr1 = character_list[0][0] # get the first item in the list. 
       tempchr2 = character_list[1][0] # get the second item in the list
       finished = tempchr1 + tempchr2 # this is the new thing we add to our list.
       character_list.remove((tempchr2, character_counts[tempchr2]))  #remove this from the list  
       character_list.remove((tempchr1,character_counts[tempchr1])) # remove this character 
       for letter in tempchr1:
            key_finished = key_finished + character_counts[letter]
            codebook[letter] = "1" + codebook[letter] ## append a 1 to the smallest probability code
       for l in tempchr2:
             key_finished = key_finished + character_counts[l]
             codebook[l] = "0" + codebook[l] ## append a 0 to the second smallest
       character_counts[finished] = key_finished
       character_list.append((finished,character_counts[finished]))
 
    return codebook

In [102]:
def encode(text, codebook):
    """Encodes a text using codebook.                                                             
                                                                                                  
    Arguments:                                                                                    
    text -- string of text to encode                                                              
    codebook -- dict that maps each symbol to a string of 0's and 1's                             
                                                                                                  
    Output:                                                                                       
    a string of 0's and 1's representing the encoded text                                         
    """
    encoded_text = ""

    # YOUR CODE HERE
    for char in text: ## look at each character in the text
        code_to_add = codebook[char] ## get its bit code
        encoded_text = encoded_text + code_to_add ## add it to the endoced string

    return encoded_text

In [113]:
def decode(encoded_text, codebook):
    """Decodes encoded text using codebook.                                                       
                                                                                                  
    Arguments:                                                                                    
    encoded_text -- a string of 0's and 1's representing the encoded text                         
    codebook -- dict that maps each symbol to a code string of 0's and 1's                        
                                                                                                  
    Output:                                                                                       
    a string representing the decoded text                                                        
    """
    decoded_text = ""

    # reverse the codebook so that it maps code strings back to symbols                           
    reverse_codebook = {val: key for key, val in codebook.items()}

    # YOUR CODE HERE
    key = ""
    for bit in encoded_text : ## get each bit in the encoded text
        key = key + bit # add it to the key variable 
        if key in reverse_codebook: ## now the key is actually in the book   
           text_to_add = reverse_codebook[key] ## look up the character that corresponds to the key
           decoded_text = decoded_text + text_to_add ## add this character to the decoded string
           key = "" ## reset the key to nothing. 
            
    return decoded_text

If you implemented the codebook, encode, and decode functions correctly, the code below should compress _War and Peace_ and return some sensible output.

In [115]:
with open("/data/warandpeace.txt", "r") as f:
    text = f.read()
  
character_counts = get_character_counts(text)
codebook = generate_codebook(character_counts)
encoded_text = encode(text, codebook)
decoded_text = decode(encoded_text, codebook)

print("First 1000 bits of encoded text: %s" + encoded_text[:1000])
print("The total length of the encoded text is %d." % len(encoded_text))
print("First 1000 characters of decoded text:\n" + decoded_text[:1000])

First 1000 bits of encoded text: %s01110001110111100010010100101111010010010000100000000110100100101110000010000100011100110100101001110000100101001001011100001010111000010000010000010001110000001010000010000100000100111100000101110001010000111111010101100010100000010000010000010001110000000111000111101110001111000010000110001011100011110111000001011100001000001011100000010000100000111101110001110110000101110001010011100011101110100010000010000010000010000010001110000101000101100010100100110100101001110001001110000100000010000000010001011110001000001000000100110100101111110111101111101010001101001010000001011100100011011110101000111011000001000010000101111100110000110001011010010100000100001000001000000110001100001100110001011000000111001100110000100100010000100011000001111010101001010011011000011110110111110100000111111010101011001011111101001100001001100101011100111000100011100000000001110001001011010101101100000001011111101000101000101110000000000110101001000101111001010010011000000

When you are done, go to the terminal and type: 

`nbgrader submit Lab-03-29 --course dlsun`

This will submit your code so that it can be graded.